# Modifications de résultats des sondages

## Import préliminaires des librairies

In [1]:
import re
import numpy as np
import pandas as pd
import locale
from locale import atof

# Read files
import PyPDF2
from pdfminer.high_level import extract_text

## Extraction utilisant une lecture de PDF

In [2]:
fp="rapport/2019-07-18.pdf"
text = extract_text(fp,page_numbers=[11])
question1="Êtes-vous  satisfait  ou  mécontent  d"
question2="Êtes-vous satisfait ou mécontent d"
print(text.find(question1))
print(text.find(question2))
print(len(question2))
text[50:]


-1
16
34


'’Edouard Philippe comme Premier \nministre ? \n\nTOTAL Satisfaits \n\n \n\n\uf0b7 Très satisfaits ....................................................  \n\n\uf0b7 Plutôt satisfaits .................................................  \n\nTOTAL Mécontents \n\n\uf0b7 Plutôt mécontents ............................................  \n\n\uf0b7 Très mécontents ...............................................  \n\n-  Ne se prononcent pas .................................  \n\n34 \n\n4 \n\n30 \n\n61 \n\n36 \n\n25 \n\n5 \n\n36 \n\n5 \n\n31 \n\n62 \n\n35 \n\n27 \n\n2 \n\nTOTAL ........................................................  \n\n100 \n\n100 \n\nRappel  \nJuin 2019 \n\nJuillet \n2019 \n\n(%) \n\n(%) \n\nVariation \n\n \n\n \n\n \n\n \n\n \n\n \n\n \n\n \n\n \n\n \n\n \n\n+2 \n\n+1 \n\n+1 \n\n+1 \n\n-1 \n\n+2 \n\n-3 \n\n \n\n \n \n \n \n \n \n \nQuestion : \n\n \n \n \n \n \n \n\n \n \n \n\nIfop pour Le Journal du Dimanche \nLes indices de popularité \uf03c Juillet 2019 \n\n10 \n\n\x0c'

In [3]:
with open(fp,"rb") as pdf_file:
        pdf_reader= PyPDF2.PdfFileReader(pdf_file)
        isEncrypted = pdf_reader.getIsEncrypted()
        nbPages= pdf_reader.numPages
        page_content=pdf_reader.getPage(11).extractText()
# print(get_base(file_path))
page_content


'Ifop\n \npour \nLe Journal du Dimanche\n \nLes indices de popularité\n \n\n \nJui\nllet\n \n2019\n \n10\n \nLes résultats\n \n \n \n \n \n \n \n \nQuestion\n \n:\n \nÊtes\n-\nvous satisfait ou mécontent \n\ncomme Premier \nministre\n \n?\n \n \n \n \n \n \n \n \nRappel \n \nJuin\n \n2019\n \nJuillet\n \n201\n9\n \n \nVariation\n \n(%)\n \n(%)\n \n \n \nTOTAL Satisfaits\n \n34\n \n36\n \n \n+2\n \n\n \nTrès satisfaits\n \n................................\n....................\n \n \n4\n \n5\n \n \n+1\n \n\n \nPlutôt satisfaits\n \n................................\n.................\n \n \n30\n \n3\n1\n \n \n+1\n \nTOTAL Mécontents\n \n61\n \n6\n2\n \n \n+1\n \n\n \nPlutôt mécontents\n \n................................\n............\n \n \n36\n \n3\n5\n \n \n-\n1\n \n\n \nTrès mécontents\n \n................................\n...............\n \n \n25\n \n27\n \n \n+2\n \n-\n \nNe se prononcent pas\n \n................................\n.\n \n \n5\n \n2\n \n \n-\n3\n \nTOTAL\n \n........

In [4]:
def get_name(filepath,page):
    question="Êtes-vous  satisfait  ou  mécontent  d"
    question2="Êtes-vous satisfait ou mécontent d"
    text = extract_text(filepath,page_numbers=[page])
    b_str=text.find(question)+len(question)+1
    if b_str==len(question):
        b_str=text.find(question2)+len(question2)+1      
    e_str=text.find(" comme")
    # si le permier caractère est un espace le supprimer
    if text[b_str]==' ' : b_str+=1
    name=text[b_str:e_str]
    name_tab=name.split()
    name_tab[1]=name_tab[1].upper()
    name=' '.join(name_tab)
    return name


In [5]:
def get_base(file_path,page=3):
    text = extract_text(file_path,page_numbers=[page])
    txt_idx="auprès d’un échantillon de "
    idx=text.find(txt_idx)+len(txt_idx)
    return float(re.findall('[0-9]+',text[idx:idx+10].replace(' ',''))[0])

In [6]:
def get_stats(filepath,page,df,name):
    with open(filepath,"rb") as pdf_file:
        pdf_reader= PyPDF2.PdfFileReader(pdf_file)
        isEncrypted = pdf_reader.getIsEncrypted()
        nbPages= pdf_reader.numPages
        page_content=pdf_reader.getPage(page).extractText()
 
    satisf_voc=['TOTAL Satisfaits','Très satisfaits','Plutôt satisfaits','TOTAL Mécontents','Plutôt mécontents','Très mécontents','Ne se prononcent pas']
    df_conv={'TOTAL Satisfaits':"ST Favorable",'Très satisfaits':"Très favorable",'Plutôt satisfaits':"Plutôt favorable",'TOTAL Mécontents':"ST Défavorable",'Plutôt mécontents':"Plutôt défavorable",'Très mécontents':"Très défavorable",'Ne se prononcent pas':"Nsp"}

    # Les catégories sont parfois séparés par un saut de ligne qu'il faut prétraité
    page_content=page_content.replace('Très \nsatisfaits','Très satisfaits')
    page_content=page_content.replace('Trè\ns satisfaits','Très satisfaits')
    page_content=page_content.replace('TOTAL \nSatisfaits','TOTAL Satisfaits')
    page_content=page_content.replace('Plutôt \nsatisfaits','Plutôt satisfaits')
    page_content=page_content.replace('Plutôt s\natisfaits','Plutôt satisfaits')
    page_content=page_content.replace('Plutôt \nmécontents','Plutôt mécontents')
    page_content=page_content.replace('P\nlutôt mécontents','Plutôt mécontents')
    page_content=page_content.replace('Ne \nse prononcent pas','Ne se prononcent pas')

    # Get all satisfaction data
    for satisf in satisf_voc:
        idx=page_content.find(satisf)+len(satisf)
        text=[x.strip() for x in page_content[idx:].split("\n")]

        i=1
        values=[]
        explore=True
        next_val_neg=False
        # code gerant l'acces tordu aux données du pdf
        while explore:
            # print(satisf)
            # print(text)
            # print(values)
            if len(text[i])==0 or ('.' in text[i]) or text[i]=='+':
                pass
            elif re.search('[a-zA-Z]', text[i]) and next_val_neg:
                values.append(0)
            else:
                if text[i]=='-':
                    if len(values)==2:
                        next_val_neg=True
                    else:
                        values.append(0)
                elif text[i]=='=':
                    values.append(0)
                else:
                    try:
                        values.append(float(text[i]))
                    except ValueError as e:
                        print(satisf)
                        print(text)
                        print(text[i])
                        print(values)
                        raise e
                    if next_val_neg:
                        values[-1]=-values[-1]
            i+=1
            # Exit condition
            if len(values)==3:
                # Check values
                if values[1]-values[0]!=values[2]: print("Values does not seems right {}".format(values))
                explore=False
        df.loc[(df.Nom==name),df_conv[satisf]]=values[1]
        # Evolutions des resultats
        if satisf == 'TOTAL Satisfaits':
            df.loc[(df.Nom==name),'Evolution ST Favorable']=values[2]
        if satisf == 'TOTAL Mécontents':
            df.loc[(df.Nom==name),'Evolution ST Défavorable']=values[2]
    return df


In [7]:
def extract_ifop_data_from_pdf(file_path,pages):
    print(file_path)
    columns=["Nom","Categorie","Groupe","Base","Très favorable","Plutôt favorable","ST Favorable","Plutôt défavorable","Très défavorable","ST Défavorable","Nsp","Evolution ST Favorable","Evolution ST Défavorable"]
    df=pd.DataFrame(columns=columns)
    # Par defaut en page 3
    base=get_base(file_path)
    for page in pages:
        name=get_name(file_path,page)
        print(base)
        new_row={'Nom':name,'Categorie':'ENSEMBLE','Groupe':'ENSEMBLE','Base':base}
        df=df.append(new_row, ignore_index=True)
        get_stats(file_path,page,df,name)
    return df

In [8]:
filename="2020-12-17"
# Enlever 1 par rapport au pdf aux pages
pages=[6,11]
df=extract_ifop_data_from_pdf("rapport/"+filename+".pdf",pages)
df['Date']=filename
df.Date=df.Date.astype('datetime64')
df[["Base","Très favorable","Plutôt favorable","ST Favorable","Plutôt défavorable","Très défavorable","ST Défavorable","Nsp","Evolution ST Favorable","Evolution ST Défavorable"]]=df[["Base","Très favorable","Plutôt favorable","ST Favorable","Plutôt défavorable","Très défavorable","ST Défavorable","Nsp","Evolution ST Favorable","Evolution ST Défavorable"]].astype(float)
# df.to_pickle('data/'+filename+'.p')
df

rapport/2020-12-17.pdf
1936.0
1936.0


,Nom,Categorie,Groupe,Base,Très favorable,Plutôt favorable,ST Favorable,Plutôt défavorable,Très défavorable,ST Défavorable,Nsp,Evolution ST Favorable,Evolution ST Défavorable,Date
0,Emmanuel MACRON,ENSEMBLE,ENSEMBLE,1936.0,5.0,33.0,38.0,32.0,28.0,60.0,2.0,-3.0,3.0,2020-12-17
1,Jean CASTEX,ENSEMBLE,ENSEMBLE,1936.0,3.0,34.0,37.0,37.0,22.0,59.0,4.0,-2.0,3.0,2020-12-17


In [16]:
import datetime
# df=pd.read_pickle('data/2021-01-08.p')
df.Date=datetime.date(2021,2,18)
df.Date=df.Date.astype('datetime64')
df.Base=1954
cols=df.columns[4:13]
df.loc[0,cols]=[6,35,41,33,23,56,3,1,-2]
df.loc[1,cols]=[3,34,37,37,21,58,5,0,0]
df
# df.to_pickle('data/2021-02-18.p')

In [10]:
# fp='data/2020-07-16.p'
# df=pd.read_pickle(fp)
# df.Date=df.Date.astype('datetime64')
# df.to_pickle(fp)

In [11]:
# val_to_change=df.iloc[1]#
# val_to_change[['Plutôt favorable',"Plutôt défavorable","Très défavorable","ST Défavorable","Evolution ST Défavorable"]]=[31,35,27,62,1]
# val_to_change
# df.iloc[1]=val_to_change
# df

In [12]:
# df.to_pickle('data/'+filename+'.p')

In [13]:
# df3=pd.concat([df,df2],ignore_index=True)
# df3.to_pickle('data/'+filename+'.p')

In [14]:
# Reading IPSOS report

# with open("rapport/2020-10-14.pdf","rb") as pdf_file:
#     pdf_report= PyPDF2.PdfFileReader(pdf_file)
#     isEncrypted = pdf_report.getIsEncrypted()
#     nbPages= pdf_report.numPages

#     page1=pdf_report.getPage(15)
#     text=page1.extractText().replace(',','.')
#     text = extract_text(pdf_file,page_numbers=[14])

# print(text)

print(df.isnull().values.any())
df.head(10)
df.describe()
df[df.Base==0]

False


,Nom,Categorie,Groupe,Base,Très favorable,Plutôt favorable,ST Favorable,Plutôt défavorable,Très défavorable,ST Défavorable,Nsp,Evolution ST Favorable,Evolution ST Défavorable,Date
